In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

from google.colab import files
import io, pandas as pd

uploaded = files.upload()   # Choose your Excel file
filename = list(uploaded.keys())[0]
print("Uploaded:", filename)

# Read Excel — detect sheet names
xls = pd.ExcelFile(io.BytesIO(uploaded[filename]))
print("Available sheets:", xls.sheet_names)

# Load first sheet (or specify a name: sheet_name='Sheet1')
df = pd.read_excel(xls, sheet_name=0)
print(" Data loaded. Shape:", df.shape)
df.head()


In [ ]:
import matplotlib.pyplot as plt

# Choose any categorical column
col = df_clean.select_dtypes(include='object').columns[0]

plt.figure(figsize=(10,5))
df_clean[col].value_counts().plot(kind='bar')
plt.xlabel(col)
plt.ylabel("Count")
plt.title("Bar Plot " )
plt.show()

In [ ]:
# STEP 2: Basic exploration
import numpy as np

print(" Shape:", df.shape)
print("\n Columns:", df.columns.tolist())
print("\n Info:")
df.info()

print("\n Missing values per column:")
print(df.isnull().sum().sort_values(ascending=False))

print("\n Unique values per column:")
print(df.nunique().sort_values(ascending=False))

df.describe(include='all').T.head(15)


In [ ]:
col = df_clean.select_dtypes(include='object').columns[0]

plt.figure(figsize=(7,7))
df_clean[col].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title("Pie Chart of " + col)
plt.ylabel("")
plt.show()

In [ ]:
df_clean = df.copy()


for c in df_clean.select_dtypes(include=["object"]).columns:
    df_clean[c] = df_clean[c].astype(str).str.strip()

# Convert columns that should be numeric to numeric, coercing errors
numeric_candidate_cols = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9"]
for col in numeric_candidate_cols:
  df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

num_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df_clean.select_dtypes(include=["object"]).columns.tolist()


for c in num_cols:
    if df_clean[c].isnull().any():
        df_clean[c] = df_clean[c].fillna(df_clean[c].median())


for c in cat_cols:
    if df_clean[c].isnull().any():
        mode_val = df_clean[c].mode().iloc[0] if not df_clean[c].mode().empty else "Unknown"
        df_clean[c] = df_clean[c].fillna(mode_val)

print(" Cleaning done.")
display(df_clean.head())

In [ ]:

date_cols = []
for c in df_clean.columns:
    sample = df_clean[c].dropna().astype(str).iloc[:5] if df_clean[c].dropna().shape[0] > 0 else []
    try:
        parsed = pd.to_datetime(sample, errors="coerce")
        if parsed.notna().sum() >= 1:
            date_cols.append(c)
    except:
        pass

print(" Date-like columns detected:", date_cols)

for c in date_cols:
    df_clean[c] = pd.to_datetime(df_clean[c], errors="coerce")
    df_clean[c+"_year"] = df_clean[c].dt.year
    df_clean[c+"_month"] = df_clean[c].dt.month
    df_clean[c+"_day"] = df_clean[c].dt.day

df_clean.head()


In [ ]:
# STEP 5: Encoding + Scaling
from sklearn.preprocessing import LabelEncoder, StandardScaler

encoded_df = df_clean.copy()
le_map = {}


small_card_cols = [c for c in cat_cols if df_clean[c].nunique() <= 10]

for c in small_card_cols:
    le = LabelEncoder()
    encoded_df[c + "_le"] = le.fit_transform(encoded_df[c].astype(str))
    le_map[c] = dict(zip(le.classes_, le.transform(le.classes_)))

print(" Label-encoded columns:", small_card_cols)


scaler = StandardScaler()
if len(num_cols) > 0:
    scaled_vals = scaler.fit_transform(encoded_df[num_cols])
    scaled_df = pd.DataFrame(scaled_vals, columns=[c + "_scaled" for c in num_cols], index=encoded_df.index)
    encoded_df = pd.concat([encoded_df, scaled_df], axis=1)
    print(" Numeric columns scaled (added with _scaled).")

encoded_df.head()


In [ ]:
num_col = df_clean.select_dtypes(include='number').columns[2]

plt.figure(figsize=(8,5))
plt.plot(df_clean[num_col])
plt.xlabel("Index")
plt.ylabel(num_col)
plt.title("Line Plot of " + num_col)
plt.show()

In [ ]:
# STEP 6: Train/Test split
from sklearn.model_selection import train_test_split

possible_targets = [c for c in encoded_df.columns if c.lower() in ("target","label","y","outcome","survived")]
target_col = possible_targets[0] if possible_targets else None

if target_col:
    X = encoded_df.drop(columns=[target_col])
    y = encoded_df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print("Split shapes:", X_train.shape, X_test.shape)
else:
    train_idx, test_idx = train_test_split(encoded_df.index, test_size=0.2, random_state=42)
    train_df = encoded_df.loc[train_idx]
    test_df = encoded_df.loc[test_idx]
    print("Index-based split:", train_df.shape, test_df.shape)


In [ ]:
# STEP 7: Save cleaned and encoded CSV files
encoded_df.to_csv("data_export_cleaned.csv", index=False)
print(" Cleaned dataset saved as data_export_cleaned.csv")


from google.colab import files
files.download("data_export_cleaned.csv")
